In [ ]:
for i, row in df.iterrows():
    in_str= row['text']
    split = re.split(r'/' , in_str)[0]
    if bool(re.search(r'\d+\.\d+$', split)) == True:
        after = re.findall(r'\d+\.\d+', split)[0]
        before = df.loc[i, 'rating_numerator']
        df.loc[i, 'rating_numerator'] = after
        print(f"The numerator in row {i} used to be {before} and now is {after}.")
    else:
        pass

In [ ]:
df_before= df.copy()
df = df[df['rating'] < 15]

#df_before= plots of the distribution of the data before and after removing the two outliers
f, axes = plt.subplots(1, 2, figsize=(12,4))

sns.boxplot(data=df_before,x= 'rating',orient='w', ax=axes[0])
axes[0].set_title('Before, Including Global Outliers')


sns.boxplot(data=df, x= 'rating',orient='w', ax=axes[1])
axes[1].set_title('After, Excluding Global Outliers');

In [ ]:
# Rounding Ratings to get a discrete scale
for i, row in df_rate.iterrows():
    before = df_rate.loc[i, 'rating']
    after = round(df_rate.rating[i], 0)
    df_rate.loc[i, 'rating'] = after
    if bool(before == after) == False :
        print(f"The rating in row {i} used to be {before} and now is {after}.")
    else:
        pass

In [ ]:
#Golden Retriever
df_gold= (df_rate[df_rate['p1']=='golden_retriever'])
df_gold= df_gold.rating.value_counts().sort_index()
df_gold = df_gold.rename_axis('rating').to_frame('gold_count')
#Labrador Retriever
df_lab= (df_rate[df_rate['p1']=='Labrador_retriever'])
df_lab= df_lab.rating.value_counts().sort_index()
df_lab = df_lab.rename_axis('rating').to_frame('lab_count')
#Pembroke
df_pem= (df_rate[df_rate['p1']=='Pembroke'])
df_pem=df_pem.rating.value_counts().sort_index()
df_pem = df_pem.rename_axis('rating').to_frame('pem_count')
#Chihuahua
df_chih= (df_rate[df_rate['p1']=='Chihuahua'])
df_chih= df_chih.rating.value_counts().sort_index()
df_chih = df_chih.rename_axis('rating').to_frame('chih_count')
#Pug
df_pug= (df_rate[df_rate['p1']=='pug'])
df_pug=df_pug.rating.value_counts().sort_index()
df_pub=df_pug.rename_axis('rating').to_frame('pug_count')

#Combining the top 5 most likley dogs
df_5dogs= pd.concat([df_pem,df_gold, df_lab,df_chih,df_pug],axis=1,sort=True).fillna(0)
df_5dogs= df_5dogs.reset_index().rename(columns={'index' : 'rating','rating' : 'pug_count'})

#Counts of all other dogs and ratings
df_oth = df_rate[(df_rate.p1 != 'golden_retriever') | (df_rate.p1 != 'Labrador_retriever') | (df_rate.p1 !='Chihuahua')| (df_rate.p1 !='Pembroke')| (df_rate.p1 !='pug')]
df_oth = df_oth.rating.value_counts().sort_index()
df_oth = df_oth.rename_axis('rating').to_frame('other_count')


#Combining the all rating counts
df_bayes= pd.merge(df_oth, df_5dogs,on='rating',how='left').fillna(0)
df_bayes['rating'] = df_bayes['rating'].astype(str)
df_bayes= df_bayes.set_index('rating',drop=True)
df_bayes.loc[:,'Total'] = df_bayes.sum(axis=1)
df_bayes.loc['Total',:] = df_bayes.sum(axis=0)

df_bayes

In [ ]:
#Deriving given calculations 
df_given = df_bayes.copy()
for col in df_given.columns:
    for i,row in df_given.iterrows():
        given = row[col]/ row['Total']
        df_given.loc[i,col] = given
#Transpositoning the matrix
df_prior = df_bayes.copy()
df_prior = df_prior.T
#Deriving prior calculations
df_pri = df_prior.copy()
for col in df_prior.columns:
    for i,row in df_prior.iterrows():
        fix = row[col]
        prior = fix / row['Total']
        df_pri.loc[i,col] = prior

#Multiplying to create probability distribution        
df_prob = df_given * df_pri.T
df_prob = df_prob.round(3)
df_prob